# Binomial Logistic Regression (Documentation Example)

The documentation example is available here: https://spark.apache.org/docs/2.1.1/ml-classification-regression.html#logistic-regression.

Binomial logistic regression is a classification algorithm, unlike linear regression. It can be used for binary classification (predicting a 1 or a 0). 

Objective: Let's see a simple example of how to run binomial logistic regression with Python and Spark. While this documentation dataset is structured/unrealistic, it provides a basic summary of how to utilise logistic regression. For a more realistic exercise, move on to the advanced logistic regression exercise. Please note that there is a huge amount of documentation available online. Ensure that you understand what you're doing and why. 

In [1]:
# Section must be included at the beginning of each new notebook. Remember to change the app name.
# If you're using VirtualBox, change the below to '/home/user/spark-2.1.1-bin-hadoop2.7'
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('logistic_regression_docs').getOrCreate()

# If you're getting an error with numpy, please type 'sudo pip3 install numpy --user' into the console.
# If you're getting an error with another package, type 'sudo pip3 install PACKAGENAME --user'. 
# Replace PACKAGENAME with the relevant package (such as pandas, etc).
from pyspark.ml.classification import LogisticRegression

In [2]:
# Load training data. Libsvm is used throughout the Spark documentation, but it's probably not relevant to your dataset. 
training = spark.read.format("libsvm").load("Datasets/sample_libsvm_data.txt")

# Print schema to get a high level view of the schema. Simply label and features. 
training.printSchema()

# Create an instance of the logistic regression model. This is where you specify features/label/prediction columns.
# The documentation example includes three parameters. We'll leave those out for now, but they're important to specify in your assignment.
lr = LogisticRegression()

# Fit the model. Note that the documentation example doesn't split the data into training/testing/validation sets.
lrModel = lr.fit(training)

# Instead of printing the coefficients/intercept, let's try to get a high level view of the model output. 
trainingSummary = lrModel.summary

# Print schema again. Note the additional columns. As with other models, we simply want to compare the label (actual value) to the predicted value.
trainingSummary.predictions.printSchema()

# Visualise the DataFrame. Visually compare label to prediction. What do you see? 
trainingSummary.predictions.show()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = true)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[19.8534775947478...|[0.99999999761359...|       0.0|
|  1.0|(692,[158,159,160...|[-20.377398194908...|[1.41321555111056...|       1.0|
|  1.0|(692,[124,125,126...|[-27.401459284891...|[1.25804865126979...|       1.0|
|  1.0|(692,[152,153,154...|[-18.862741612668...|[6.42710509170303...|       1.0|
|  1.0|(692,[151,152,153...|[-20.483011833009...|[1.27157209200604...|       1.0|
|  0.0|(692,[129,130,131...|[19.8506078990277...

## Bonus: Train/Test Split

While this isn't part of the documentation example, it's important to understand how to do a train/test split in order to evaluate the model's results.

In [3]:
# Let's split the data into a training/testing split so we can evalaute the model.
lr_train,lr_test = training.randomSplit([0.7,0.3])

# Create instance of the logistic regression model.
final_model = LogisticRegression()

# Now fit the model on a subset of data.
fit_final = final_model.fit(lr_train)

# And evaluate it against the test data.
predictions_and_labels = fit_final.evaluate(lr_test)

predictions_and_labels.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[95,96,97,12...|[22.7469390876039...|[0.99999999986783...|       0.0|
|  0.0|(692,[121,122,123...|[19.5758692957412...|[0.99999999685001...|       0.0|
|  0.0|(692,[122,123,124...|[16.9339400170149...|[0.99999995577342...|       0.0|
|  0.0|(692,[123,124,125...|[34.1630647521288...|[0.99999999999999...|       0.0|
|  0.0|(692,[123,124,125...|[30.1535882634884...|[0.99999999999991...|       0.0|
|  0.0|(692,[124,125,126...|[28.6898147896084...|[0.99999999999965...|       0.0|
|  0.0|(692,[124,125,126...|[24.3959410526453...|[0.99999999997459...|       0.0|
|  0.0|(692,[124,125,126...|[21.6637401587347...|[0.99999999960955...|       0.0|
|  0.0|(692,[125,126,127...|[25.7020255947206...|[0.99999999999311...|       0.0|
|  0.0|(692,[126

## Logistic Regression Evaluation Metrics

Evaluators are an important when working with machine learning, let's see some basics for Logistic Regression. Check out these links:

https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.evaluation.BinaryClassificationEvaluator

https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.evaluation.MulticlassClassificationEvaluator

For a binary evaluator, you can get the area under the ROC curve or the area under the precision/recall curve.

For a multi-class evalautor, you can get back accuracy, precision/recall, etc. 

In [4]:
# Let's import one evalulation metrics. 
# Remember, binary is for true and false (0 and 1) predictions, while multi-class is for multiple classification classes.
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [5]:
# Create an instance of the evaluator (finds area under the curve).
evaluator = BinaryClassificationEvaluator()

# Evaluate the predictions. 
my_final_roc = evaluator.evaluate(predictions_and_labels.predictions)

# Display the results. 
my_final_roc

# According to this evaluation metric, the area under the curve is 1.0. A perfect fit? Is that realistic?  

1.0

Note that the unrealistic evaluation metric results are because of the test dataset. Move on to the Advanced Logistic Regression lab to start using a more realistic dataset. 

Applying Decision Trees is similar to applying Logistic Regression. Find out more here: https://spark.apache.org/docs/2.1.1/ml-classification-regression.html#decision-tree-classifier